<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/Penguin_%EB%B6%84%EB%A5%98%2C_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Penguin Dataset
- https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv

In [26]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
# pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

##### 데이터 불러오기
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [27]:
df = df.dropna()

In [28]:
df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex'],
      dtype='object')


1. 분류, Y = isMale
- 분류 분석을 하기 위해 데이터를 7:3으로 split하기

In [45]:
# 불필요 변수
COL_DEL = []
# 수치형 변수
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
# 범주형 변수 
COL_CAT = ['species','island']
# 종속 변수
COL_Y = ['isMale']

df_X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g','species','island']]
df['isMale'] = 0
df.loc[df.sex == 'MALE', 'isMale'] = 1
df_Y = df[['isMale']]

from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df_X, df_Y, test_size=0.3)

X_train_raw.reset_index(drop=True, inplace=True)
X_test_raw.reset_index(drop=True, inplace=True)
y_train_raw.reset_index(drop=True, inplace=True)
y_test_raw.reset_index(drop=True, inplace=True)

X_raw = pd.concat([X_train_raw, X_test_raw]) #위 아래로 합치기,
# Scaling 및 Onehot encoding을 위한 데이터
train_raw =  pd.concat([X_train_raw, y_train_raw], axis=1) # 왼쪽 오른쪽으로 합치기
# raw를 지우는 데이터 처리를 위한 데이터

In [46]:
# X_train_raw.head()
# print(X_train_raw.columns)
# print(X_train_raw.dtypes)
print(X_train_raw.info()) # 결측치 확인
# X_train_raw.describe() #이상치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bill_length_mm     233 non-null    float64
 1   bill_depth_mm      233 non-null    float64
 2   flipper_length_mm  233 non-null    float64
 3   body_mass_g        233 non-null    float64
 4   species            233 non-null    object 
 5   island             233 non-null    object 
dtypes: float64(4), object(2)
memory usage: 11.0+ KB
None


In [47]:
##### 불필요 변수 제외
# X_train = X_train_raw.iloc[:, 1:]
# y_train = y_train_raw.iloc[:, 1:]
# X_test = X_test_raw.iloc[:, 1:]

##### 불필요 변수 제외X
X_train = X_train_raw.copy()
y_train = y_train_raw.copy()
X_test = X_test_raw.copy()

In [48]:
##### 결측치 처리하기
# from sklearn.impute import SimpleImputer
# # imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )

In [49]:
##### 수치형 변수 - 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_raw[COL_NUM])
X_train[COL_NUM]=scaler.transform(X_train[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

In [50]:
##### 범주형 변수 - 그 다음 원핫 인코딩 - 여기서는 제외했으므로 사용하지 않음
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X_train[COL_CAT])
# numpy array로 변환
X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

# 다시 DF화 하기
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names())
# print(X_train_ohe.head())

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

In [52]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

In [53]:
##### 모델학습하기

# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
# C 값이 작을 수록 규제가 커지고 일반화
modelLR = LogisticRegression(C=0.1, random_state=0)
modelLR.fit(X_tr, y_tr)

# KNN 분류
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr)

# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators=100, max_depth=10)
modelRF.fit(X_tr, y_tr)

# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

# XGBoost
from xgboost import XGBClassifier
modelX = XGBClassifier(n_estimators=100, learning_rate=0.02, max_depth=10)
xgb_model = modelX.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='logloss',eval_set=[(X_val, y_val)])

# 이미 결정해버림
y_val_pred = modelLR.predict(X_val)
# [0일 확률, 1일 확률]확률값
y_val_pred_proba = modelLR.predict_proba(X_val)

[0]	validation_0-logloss:0.681452
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.67006
[2]	validation_0-logloss:0.659083
[3]	validation_0-logloss:0.648638
[4]	validation_0-logloss:0.638432
[5]	validation_0-logloss:0.628588
[6]	validation_0-logloss:0.619219
[7]	validation_0-logloss:0.610045
[8]	validation_0-logloss:0.601314
[9]	validation_0-logloss:0.592749
[10]	validation_0-logloss:0.584679
[11]	validation_0-logloss:0.576749
[12]	validation_0-logloss:0.568982
[13]	validation_0-logloss:0.56168
[14]	validation_0-logloss:0.554397
[15]	validation_0-logloss:0.547871
[16]	validation_0-logloss:0.541031
[17]	validation_0-logloss:0.534474
[18]	validation_0-logloss:0.528549
[19]	validation_0-logloss:0.522318
[20]	validation_0-logloss:0.516456
[21]	validation_0-logloss:0.509173
[22]	validation_0-logloss:0.502106
[23]	validation_0-logloss:0.495246
[24]	validation_0-logloss:0.488073
[25]	validation_0-logloss:0.481373
[26]	validation_0-logloss:0.47507

In [54]:
##### 평가하기
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, classification_report, mean_squared_log_error

# confusion_matrix
# score = confusion_matrix(y_val.iloc[:,0], y_val_pred_proba)

# classification_report
# score = classification_report(y_val, y_val_pred_proba)

# mean_squared_log_error
# score = mean_squared_log_error(y_val, y_val_pred_proba)

# roc_auc_score
score = roc_auc_score(y_val, y_val_pred_proba[:, 1])

print( score )

0.9853061224489796


2. 회귀, Y = body_mass_g
- 회귀 분석을 하기 위해 데이터를 7:3으로 split하기

In [55]:
# 불필요 변수
COL_DEL = []
# 수치형 변수
COL_NUM = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
# 범주형 변수 
COL_CAT = ['species','island', 'sex']
# 종속 변수
COL_Y = ['body_mass_g']

df_X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm','species','island', 'sex']]
df_Y = df[['body_mass_g']]

from sklearn.model_selection import train_test_split
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df_X, df_Y, test_size=0.3)

X_train_raw.reset_index(drop=True, inplace=True)
X_test_raw.reset_index(drop=True, inplace=True)
y_train_raw.reset_index(drop=True, inplace=True)
y_test_raw.reset_index(drop=True, inplace=True)

X_raw = pd.concat([X_train_raw, X_test_raw]) #위 아래로 합치기,
# Scaling 및 Onehot encoding을 위한 데이터
train_raw =  pd.concat([X_train_raw, y_train_raw], axis=1) # 왼쪽 오른쪽으로 합치기
# raw를 지우는 데이터 처리를 위한 데이터

In [56]:
# X_train_raw.head()
# print(X_train_raw.columns)
# print(X_train_raw.dtypes)
print(X_train_raw.info()) # 결측치 확인
# X_train_raw.describe() #이상치 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bill_length_mm     233 non-null    float64
 1   bill_depth_mm      233 non-null    float64
 2   flipper_length_mm  233 non-null    float64
 3   species            233 non-null    object 
 4   island             233 non-null    object 
 5   sex                233 non-null    object 
dtypes: float64(3), object(3)
memory usage: 11.0+ KB
None


In [58]:
##### 불필요 변수 제외
X_train = X_train_raw
y_train = y_train_raw
X_test = X_test_raw

In [ ]:
##### 결측치 처리하기
# from sklearn.impute import SimpleImputer
# # imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )

In [59]:
##### 수치형 변수 - 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_raw[COL_NUM])
X_train[COL_NUM]=scaler.transform(X_train[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

In [60]:
##### 범주형 변수 - 그 다음 원핫 인코딩 - 여기서는 제외했으므로 사용하지 않음
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X_raw[COL_CAT])
# numpy array로 변환
X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

# 다시 DF화 하기
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names())
# print(X_train_ohe.head())

X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

In [61]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

In [62]:
##### 모델학습하기

# sklearn 선형 회귀
from sklearn.linear_model import LinearRegression
# C 값이 작을 수록 규제가 커지고 일반화
modelLR = LinearRegression()
results = modelLR.fit(X_tr, y_tr)

# statsmodel 선형 회귀
import statsmodels.api as sm 

X_tr2 = sm.add_constant(X_tr)
model2 = sm.OLS(y_tr, X_tr2)
results = model2.fit()

# 이미 결정해버림
y_val_pred = modelLR.predict(X_val)

In [65]:
#### 회귀계수 해석
# 절편
print(modelLR.intercept_)
# 기울기
coef = pd.Series(data=modelLR.coef_[0], index= X_train_fin.columns)
print(coef.sort_values())
print("***********************************************************")
print(results.summary())

[4141.48303367]
x0_Chinstrap        -455.263930
x0_Adelie           -251.331275
x2_FEMALE           -195.519275
x1_Torgersen         -61.313526
x1_Dream              22.289288
x1_Biscoe             39.024237
bill_length_mm        75.836031
bill_depth_mm        108.713244
x2_MALE              195.519275
flipper_length_mm    232.955624
x0_Gentoo            706.595204
dtype: float64
***********************************************************
                            OLS Regression Results                            
Dep. Variable:            body_mass_g   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     109.4
Date:                Sun, 06 Feb 2022   Prob (F-statistic):           1.48e-59
Time:                        12:08:42   Log-Likelihood:                -1163.5
No. Observations:                 163   AIC:                             2345.
Df R

In [66]:
##### 평가하기
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)                 # 1. mse
rmse = mean_squared_error(y_val, y_val_pred, squared=False) # 2. rmse : np.sqrt(mse)도 가능

r2 = r2_score(y_val, y_val_pred)                            # 3. r2
score = modelLR.score(X_val, y_val)                         # 이 것 또한 R2

print(score)
print('MSE : {0:.3f} , RMSE : {1:.3F} '.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2))

0.9030666248392861
MSE : 62100.181 , RMSE : 249.199 
Variance score : 0.903
